In [10]:
import pandas as pd
import numpy as np
import copy
from sklearn.model_selection import train_test_split
import cProfile


import sys
from pathlib import Path

# Go up one directory to get to master/
project_root = str(Path.cwd().parent)
sys.path.append(project_root)

from helpers.helpers import preprocess_dataframes

In [11]:
def gini_index(arr: np.array,
               instance_size: int,
               K: list,
               y_idx: int = 0,
               weighted: bool = True):
    """

    :param arr: The subset of data to evaluate (rows = instances, columns = features/labels).
    :param instance_size: The total number of instances in the original dataset (used for weighting)
    :param K: list — The list of unique class labels
    :param y_idx: y_idx: int (default 0) — The index of the column in arr that contains the class labels.
    :param weighted: Whether to weight the Gini index by the proportion of the subset size to the total dataset size
    :return:
    """
    sum_ = 0
    for k in K:
        sum_ += np.power(len(arr[np.where(arr[:, y_idx] == k)]) / len(arr), 2) # arr[np.where(arr[:, y_idx] == k)] selects all rows where the class label is k, len(...) / len(arr) computes the proportion of class k in the subset
    sum_ = 1 - sum_
    if weighted:
        sum_ = (len(arr) / instance_size) * sum_
    return sum_

In [12]:
def calculate_gini_old(data: pd.DataFrame,
                   P: list,
                   K: list,
                   nodes: dict) -> dict:
    """

    :param data: pd.DataFrame — The dataset
    :param P: list Indices of features to consider
    :param K: lsit Unique class labels
    :param nodes: dict — Contains: "leaf_nodes": list of leaf node identifiers. leaf_nodes_path": dict mapping each leaf node to a path (list of values for features).
    :return:
    """
    df_arr = np.array(data)
    n = len(data) # total number of instances.
    gini_dict = dict()
    for leaf_ in nodes["leaf_nodes"]:
        temp = dict() # store Gini values for this leaf
        first_var = nodes["leaf_nodes_path"][leaf_][0] #{'leaf_nodes': [4, 5, 6, 7], 'leaf_nodes_path': {4: [1, 1], 5: [1, 0], 6: [0, 1], 7: [0, 0]}}
        second_var = nodes["leaf_nodes_path"][leaf_][1] #first_var, second_var — the first two values in the path to this leaf (used as feature values for filtering)
        for feature_i in P:
            arr = df_arr[np.where((df_arr[:, feature_i] == first_var))] #Selects rows where feature feature_i equals first_var.

            for feature_j in P:
                arr_2 = arr[np.where(arr[:, feature_j] == second_var)] #Further filters to rows where feature feature_j equals second_var
                if len(arr_2) > 0: #Calculate Gini index for arr_2 (so for all rows matching (1, 0; having/not having feature i) the decision variables)
                    temp[feature_i, feature_j] = gini_index(arr=arr_2,
                                                            instance_size=n,
                                                            K=K,
                                                            weighted=True)
        gini_dict[leaf_] = copy.deepcopy(temp)
        del temp
        del arr

    return gini_dict


In [13]:
def gini_index_fast(y):
    """Vectorized Gini calculation for a 1D array of labels."""
    _, counts = np.unique(y, return_counts=True) #returns the unique labels in y and how many times each appears, counts is an array of the number of occurrences for each unique label
    probs = counts / len(y)
    return 1 - np.sum(probs ** 2)
    #probs = counts / counts.sum()
    #if counts.sum() == len(y):
        #return 1 - np.sum(probs ** 2)

def calculate_gini_fast(data: pd.DataFrame, P: list, K: list, nodes: dict) -> dict:
    """
    :param data: pd.DataFrame — The dataset
    :param P: list of Indices of features to consider
    :param K: list Unique class labels
    :param nodes: dict — Contains: "leaf_nodes": list of leaf node identifiers. leaf_nodes_path": dict mapping each leaf node to a path (list of values for features).
    :return:
    """
    gini_dict = dict()
    n = len(data) #Total number of rows in the dataset
    for leaf_ in nodes["leaf_nodes"]:
        temp = dict()
        #These are the feature values that define the path to this leaf (e.g., in a decision tree, the values that must be matched to reach this leaf):
        first_var = nodes["leaf_nodes_path"][leaf_][0] #{'leaf_nodes': [4, 5, 6, 7], 'leaf_nodes_path': {4: [1, 1], 5: [1, 0], 6: [0, 1], 7: [0, 0]}}
        second_var = nodes["leaf_nodes_path"][leaf_][1] #first_var, second_var — the first two values in the path to this leaf (used as feature values for filtering)
        for feature_i in P:
            arr = data[data.iloc[:, feature_i] == first_var] # filters to rows where feature_i equals first_var
            for feature_j in P:
                arr_2 = arr[arr.iloc[:, feature_j] == second_var] #Further filters arr to rows where feature_j equals second_var
                if not arr_2.empty:
                    gini = gini_index_fast(arr_2.iloc[:, 0].values) #Calculates the Gini impurity of the labels in the first column (assumed to be the label column).
                    weighted_gini = (len(arr_2) / n) * gini
                    temp[(feature_i, feature_j)] = weighted_gini
        gini_dict[leaf_] = temp
    return gini_dict

In [14]:
def gini_for_leaf(args):
    leaf_, df_arr, n, P, K, nodes = args
    temp = dict()
    first_var = nodes["leaf_nodes_path"][leaf_][0]
    second_var = nodes["leaf_nodes_path"][leaf_][1]
    for feature_i in P:
        arr = df_arr[np.where((df_arr[:, feature_i] == first_var))]
        for feature_j in P:
            arr_2 = arr[np.where(arr[:, feature_j] == second_var)]
            if len(arr_2) > 0:
                temp[feature_i, feature_j] = gini_index(
                    arr=arr_2, instance_size=n, K=K, weighted=True
                )
    return (leaf_, temp)

In [15]:
from multiprocessing import Pool, cpu_count

def calculate_gini(data: pd.DataFrame, P: list, K: list, nodes: dict, amount_cores = 1) -> dict:
    df_arr = np.array(data)
    n = len(data)
    # Prepare arguments for each process
    args = [
        (leaf_, df_arr, n, P, K, nodes)
        for leaf_ in nodes["leaf_nodes"]
    ]
    with Pool(processes=amount_cores) as pool:
        results = pool.map(gini_for_leaf, args)
    # Combine results into a dictionary
    gini_dict = {leaf_: temp for leaf_, temp in results}
    return gini_dict

In [16]:
from rolling_lookahead_dt_pulp.oct.tree import generate_nodes
from pulp import *

def generate_model_pulp(
        P: list,
        K: list,
        data: pd.DataFrame,
        y_idx: int = 0,
        time_limit: float = 1800, # moved to train_model to give parameter to solver directly
        gap_limit: float = None,
        log_to_console: bool = False,
        big_m: int = 99,
        amount_cores = 1,
        test = False,
        criterion: str = "gini",
):
    """

    :param criterion:
    :param big_m:
    :param depth:
    :param P:
    :param K:
    :param data:
    :param leaf_nodes_path:
    :param y_idx:
    :param time_limit:
    :param gap_limit:
    :param log_to_console:
    :return:
    """

    # Create parent & leaf nodes

    leaf_nodes_path = {4: [1, 1], #dict for all leafes in depth 2 tree and der respective split conditions
                       5: [1, 0],
                       6: [0, 1],
                       7: [0, 0]}
    depth = 2
    parent_nodes, leaf_nodes = generate_nodes(depth) # (for depth 2) returns [1,2,3] and [4,5,6,7]

    nodes = dict()
    nodes["leaf_nodes"] = leaf_nodes
    nodes["leaf_nodes_path"] = leaf_nodes_path

    print(nodes)

    if criterion == "gini":
        logging.info("Calculating gini..")
        #coef_dict = calculate_gini(data=data, P=P, K=K, nodes=nodes, amount_cores = amount_cores)
        if test == False:
            coef_dict = calculate_gini_fast(data=data, P=P, K=K, nodes=nodes)
            #with open(f'./result_gini_fast.txt', 'w') as f:
                #f.write(str(coef_dict))
        else:
            coef_dict = calculate_gini_old(data=data, P=P, K=K, nodes=nodes)
            #with open(f'./result_gini_old.txt', 'w') as f:
                #f.write(str(coef_dict))

        
    elif criterion == "misclassification":
        logging.info("Calculating misclassification..")
        coef_dict = calculate_misclassification(data=data,
                                                P=P,
                                                nodes=nodes)
    # init model
    model = LpProblem("RollOCT", LpMinimize) # Sets the objective to be minimized

    # # x[i,j] and y[i,k] as binary variables
    x = dict()
    for i in P:
        for j in P:
            x[i, j] = LpVariable(f'x[{i},{j}]', cat='Binary') #creates variable
            # In PuLP, variables are automatically added to the model when you include them in constraints or the objective function—you do not need to explicitly register them with the model
            # cat='Binary': Specifies the variable type as binary
            # name=f'x[{i}, {j}]': Assigns a name to the variable for easier identification and debugging
            # x[i, j]: Binary variable indicating whether feature i is used for the first split and feature j for the second split

    y = dict()
    for i in P:
        for k in P:
            y[i, k] = LpVariable(f'y[{i},{k}]', cat='Binary')

#lpSum is to gulps what quicksum is to gurobi

    # Constraint 1: Exactly one (i,j) pair is selected
    model += lpSum(x[i,j] for i in P for j in P) == 1, "C1b" # Ensures that exactly one combination of features (i, j) is selected for the first two splits (criteria (1b))
    # Constraint 2: Exactly one (i,k) pair is selected
    model += lpSum(y[i,k] for i in P for k in P) == 1, "C1c"  # implements criteria (1c) same way as above
    # Constraint 3
    for i in P:
        model += lpSum(x[i,j] for j in P) == lpSum(y[i,k] for k in P), f"C1d_{i}" # Links the x and y variables, ensuring that for each feature i, the sum of x[i, j] across j equals the sum of y[i, k] across k. This ensures consistency between the splits (criteria (1d))

    # Constraint 4
    # add big m; Acts as a penalty for invalid splits (if a combination is not present in coef_dict, it uses big_m as a large penalty)

# implements criteria (1a)
# variable x[i, j], y[i, k] is binary, so 1 or 0. So the multiplication makes sense
# .get(key, default): Dictionary method to safely retrieve values, using big_m as a fallback
# coef_dict[4].get((i, j))
# in coef_dict[4].get((i, j) is value of loss function of leaf 4 with features i,j from P. Value is found in dict if it matches the respective split condition of leaf 4 [1, 1]. If not penalty big m is used
# => damit der Wert fuer loss function bei feature combination i,j im coef_dict steht, müssen i,j (nach definition wie coef_dict erstellt wird) dort zur split condition (vorhanden/nicht vorhanden) gematched haben  
# => also man kann in coef_dict[4].get((i, j, big_m) nur Werte für die features finden, die im Datensatz der Kombination [1,1] entsprochen haben. Falls der Wert nicht vorhanden ist (alle i,j Kombinationen werden iteriert), wird stattdessen Wert big_m genommen

    obj = lpSum(
        (coef_dict[4].get((i, j), big_m) + coef_dict[5].get((i, j), big_m)) *
        x[i, j]
        for i in P
        for j in P) + \
          lpSum((coef_dict[6].get((i, k), big_m) + coef_dict[7].get((i, k), big_m)) *
                   y[i, k]
                   for i in P
                   for k in P)

    model += obj, "Objective"


    # #set a time limit (if using a compatible solver like CBC or Gurobi):
    # if time_limit:
    #     m.setParam("TimeLimit", time_limit)
    #     logging.info(f'Setting Time Limit as {time_limit}')

    # if gap_limit is not None:
    #     m.setParam("MipGap", gap_limit) #MipGap: Sets the optimality gap tolerance for early termination
    #     logging.info(f'Setting Optimality Gap as {gap_limit}')

    # m.setParam("LogToConsole", int(log_to_console))
    # logging.info(f'Setting LogToConsole as {log_to_console}')
    # m.update() #Updates the model with all changes.

    model_dict = {
        'model': model,
        'params': {
            'var_x': x,
            'var_y': y,
            'y_idx': y_idx
        },
        'nodes': {
            'leaf_nodes': leaf_nodes,
            'parent_nodes': parent_nodes,
            "leaf_nodes_path": leaf_nodes_path,
        },
        'depth': depth,
        "P": P,
        "K": K
    }
    logging.info('Model generation is done.')

    return model_dict , coef_dict
    

In [ ]:
from deepdiff import DeepDiff

if __name__ == "__main__":
    # Parameter aus dem Aufrug von run
    depth = 8
    time_limit = 1800
    criterion = "gini"
    amount_cores = 7

    dataset_name = 'adult'

    if dataset_name == 'test':
    # Load your training and test datasets
        data = pd.read_csv("data/stacked.csv")

    if dataset_name == 'adult':
    # Load your training and test datasets
        data = pd.read_csv("data/adult/stacked.csv")

    X = data.drop(columns=['y'])  # All columns except the target
    y = data['y']                 # Only the target column

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=100, stratify=y, random_state=42)

    stacked_train = pd.concat([y_train, X_train], axis=1, ignore_index=False)
    stacked_test = pd.concat([y_test, X_test],axis=1, ignore_index=False)

    train_data = stacked_train
    test_data = stacked_test

    feature_columns = train_data.columns[1:]

    train, test = preprocess_dataframes( #./rollo_oct/utils/helpers.py
        train_df=train_data,
        test_df=test_data,
        target_label="y",
        features=feature_columns)

    df = pd.concat([train, test])
    P = [int(i) for i in
         list(train.loc[:, train.columns != 'y'].columns)]
    train.columns = ["y", *P]
    test.columns = ["y", *P]
    K = sorted(list(set(df.y)))

    #profiler = cProfile.Profile()
    #profiler.enable()
    #main_model_fast, coef_dict_fast = generate_model_pulp(P=P, K=K, data=train, y_idx=0, big_m=99, criterion=criterion, amount_cores = amount_cores)
    main_model_old, coef_dict_old = generate_model_pulp(P=P, K=K, data=train, y_idx=0, big_m=99, criterion=criterion, amount_cores = amount_cores, test=True)
    #profiler.disable()
    #profiler.dump_stats("profile_gini_preprocess.out")

    #print(coef_dict_fast == coef_dict_old)
    #diff = DeepDiff(coef_dict_fast, coef_dict_old)
    #print(diff)

    #gini_fast: 3m5.2s
    #gini_old: 4m46.9s

    # time default adult: 4min51.1s
    # time multithreaded adult: 4m 27.8s, 4m21.3s

{'leaf_nodes': [4, 5, 6, 7], 'leaf_nodes_path': {4: [1, 1], 5: [1, 0], 6: [0, 1], 7: [0, 0]}}
